In [1]:
from util import *
start_time = (pd.datetime.now(pytz.utc)-pd.DateOffset(-1)).strftime('%Y-%m-%d 00:00:00')
end_time = (pd.datetime.now(pytz.utc)-pd.DateOffset(-2)).strftime('%Y-%m-%d 23:00:00')
utc_time = pd.date_range(start=start_time, end=end_time, freq="1H")

dfs = []
for city in ['bj', 'ld']:
    df_aq = pd.read_csv("../input/" + city + "_aq_imputed.csv")
    df_aq.utc_time = pd.to_datetime(df_aq.utc_time)
    df_aq_table = get_aq_stats(df_aq)
    df_aq_table = df_aq_table.reset_index()
    df_station = pd.read_csv("../input/" + city +"_aq_stations.csv")
    df_sample = get_submission_init(df_station, city)
    df_sample['weekofyear'] = df_sample.utc_time.dt.weekofyear
    df_sample['hour'] = df_sample.utc_time.dt.hour
    df = pd.merge(df_sample, df_aq_table, how='left', suffixes=("", "_stats"),
                  left_on =['stationId', 'weekofyear', 'hour'], 
                  right_on=['stationId', 'weekofyear', 'hour'])

    time_to_idx = {time:str(i) for i, time in enumerate(utc_time)}
    df['test_id'] = df.stationId + "#" + df.utc_time.apply(lambda time: time_to_idx[time])
    if city=='ld':
        df['O3'] = 0
    df = df[['test_id', 'PM2.5', 'PM10', 'O3']].sort_values(by='test_id')
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

mmdd = pd.datetime.now(pytz.utc).strftime('%m%d')
filename = "iar_"+ mmdd + "_stats.csv"
df.to_csv("../submission/"+ filename, index=False)
print(filename, start_time, end_time)

iar_0531_stats.csv 2018-06-01 00:00:00 2018-06-02 23:00:00


In [3]:
kdd_2018_submit(filename, "Stats baseline")

{"success": true}
